# Deploy VectorDB and process documents

## Setup vectordb

In [1]:
"""import module"""
import pandas as pd
import os
from collections import Counter

from calyx.vectordb.vectordb import VectorDB
from calyx.config import settings
from calyx.vectordb.helper import create_id

/Users/almagribisadli/miniconda/envs/shopee/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""Load vectordb object"""
VectorDB = VectorDB()

In [3]:
"""load dataset"""
DB_DIR = "data"
fpath = os.path.join(DB_DIR, "movie_description.csv")
df = pd.read_csv(fpath)

In [4]:
"""convert df to dict"""
rows = df.to_dict("records")

In [5]:
"""embedd text and store in vectordb"""
def store_document(rows:dict, name_col:str, text_col:str, fpath:str):
    stats = Counter()
    for row in rows:
        try:
            VectorDB.add_document(doc_id = create_id(row[name_col]),
                                    text = row[text_col],
                                    text_col = text_col,
                                    metadata=row)
            stats["success"]+=1
        except:
            stats["error"]+=1
    dbpath = os.path.join(DB_DIR, f"{os.path.basename(fpath).split('.')[0]}.json")
    VectorDB.save(dbpath)
    return stats, dbpath

In [6]:
"""call store document function"""
stats, dbpath = store_document(rows=rows, name_col="Movie Name", text_col="Description", fpath=fpath)
print("=== Processing Summary ===")
for key, value in stats.items():
    print(f"{key.capitalize():<10}: {value}")
print(f"vector store as {dbpath} database")

=== Processing Summary ===
Success   : 31
vector store as data/movie_description.json database


In [7]:
"""load db"""
dbname = "data/movie_description.json"
VectorDB.load(dbname)

In [8]:
"""test query"""
question = "Give me movie list about intergalactic wars"
result = VectorDB.search(query=question, top_k=5)
for i in result:
    print(i)

{'id': '4aed55d7-ab7b-5c03-a39d-6a9b30ad15bc', 'text': "Luke Skywalker joins forces with a Jedi Knight, a cocky pilot, a Wookiee and two droids to save the galaxy from the Empire's world-destroying battle station, while also attempting to rescue Princess Leia from the mysterious Darth Vader.", 'metadata': {'Movie Name': 'Star Wars', 'Watch Time': '121 min', 'Movie Rating': 8.6, 'Meatscore of movie': 90, 'Votes': '34,709', 'Gross': '$322.74M'}, 'score': 0.3356850053406776}
{'id': '90a1b6cc-3eed-513e-837e-5e3cfaf3c14a', 'text': 'After the Rebels are brutally overpowered by the Empire on the ice planet Hoth, Luke Skywalker begins Jedi training with Yoda, while his friends are pursued across the galaxy by Darth Vader and bounty hunter Boba Fett.', 'metadata': {'Movie Name': 'Star Wars: Episode V - The Empire Strikes Back', 'Watch Time': '124 min', 'Movie Rating': 8.7, 'Meatscore of movie': 82, 'Votes': '34,709', 'Gross': '$290.48M'}, 'score': 0.2872858616152044}
{'id': 'e766938f-d53b-58f1-